In [5]:
# LSTM Training for Next Word Prediction
#This notebook implements an LSTM model for next-word prediction using Shakespeare's 'Hamlet' text dataset. The process includes data preparation, model training, and evaluation.


In [6]:
import nltk
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
 #Enable Mixed Precision Training for best gppu utilaization in macOS

from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [8]:
 #Download the Gutenberg dataset
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sanatankhemariya/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [9]:
# Load Shakespeare's Hamlet
from nltk.corpus import gutenberg
data = gutenberg.raw('shakespeare-hamlet.txt')

In [10]:
with open('shakespeare.txt', 'w') as file:
    file.write(data)


In [11]:
#Load and Preprocess Text Data

# Load the text data
with open('shakespeare.txt') as file:
    text = file.read().lower()

# Tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(f"Total Words: {total_words}")

Total Words: 4818


In [ ]:
#Generate Input Sequences

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)


In [12]:

# Padding the sequences
max_sequence_len = max(len(x) for x in input_sequences)
input_seq = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [13]:
# Splitting predictors (x) and label (y)
x, y = input_seq[:, :-1], input_seq[:, -1]
y = to_categorical(y, num_classes=total_words)

In [15]:
# Split into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [16]:
# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [19]:
# Define the Model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len - 1))  # Specify input shape
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

#Compile the Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [4]:

model.summary()

# Train the Model
history = model.fit(
    x_train, y_train,
    epochs=100,  #Reduce epochs for testing (increase based on needs)
    batch_size=32,  #Smaller batch size for memory efficiency
    validation_data=(x_test, y_test),
    verbose=1,
    callbacks=[early_stopping]
)

# Save the Model
model.save('shakespeare_lstm_model.h5')

# Test GPU Utilization
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sanatankhemariya/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Total Words: 4818


/opt/homebrew/Caskroom/miniconda/base/envs/tf_macos/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-12-23 13:54:33.282934: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-12-23 13:54:33.282981: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-23 13:54:33.282988: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-23 13:54:33.283019: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-23 13:54:33.283035: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bu

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


2024-12-23 13:54:34.278580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.0264 - loss: 7.2341 - val_accuracy: 0.0336 - val_loss: 6.7040
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.0367 - loss: 6.4801 - val_accuracy: 0.0422 - val_loss: 6.7525
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.0421 - loss: 6.3625 - val_accuracy: 0.0484 - val_loss: 6.7563
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.0482 - loss: 6.2484 - val_accuracy: 0.0507 - val_loss: 6.7626
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.0533 - loss: 6.0710 - val_accuracy: 0.0560 - val_loss: 6.7566
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0602 - loss: 5.9305 - val_accuracy: 0.0637 - val_loss: 6.7717
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0729 - loss: 5.8077 - val_accuracy: 0.0647 - val_loss: 6.7736
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0825 - loss: 5.6530 - val_

Num GPUs Available: 1


In [82]:
# Function to predict the next word with session clearing
from tensorflow.keras.backend import clear_session
def predict_next_word_safe(model, tokenizer, text, max_sequence_len):
    clear_session()
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

# Function to predict the next sequence of words with session clearing
def predict_next_words_safe(model, tokenizer, text, max_sequence_len, num_words=6, temperature=0.6):
    clear_session()
    generated_words = []
    current_text = text

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([current_text])[0]
        if len(token_list) >= max_sequence_len:
            token_list = token_list[-(max_sequence_len-1):]
        padded_sequence = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(padded_sequence, verbose=0)[0]
        predictions = np.clip(np.log(predictions) / temperature, 1e-8, 1.0)
        exp_predictions = np.exp(predictions - np.max(predictions))
        probabilities = exp_predictions / np.sum(exp_predictions)
        predicted_index = np.random.choice(len(probabilities), p=probabilities)
        output_word = None
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        if output_word:
            generated_words.append(output_word)
            current_text += " " + output_word
        else:
            break

    return current_text

# Example usage
input_text = "shall we open up"
print(f"Input text: {input_text}")
max_sequence_len = max_sequence_len
next_word = predict_next_word_safe(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

# Sentence prediction example
input_text = "in the name of the lord"
result = predict_next_words_safe(model, tokenizer, input_text, max_sequence_len)
print(f"Input text: {input_text}")
print(f"Prediction: {result}")


Input text: shall we open up
Next Word Prediction: sorrow
Input text: in the name of the lord
Prediction: in the name of the lord auoyd drift limed naught beards beckons


Increase the number of epochs for more accuracy ..


In [83]:
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [84]:
# Save the Model
model.save('shakespeare_lstm_model.keras')  # Saves in TensorFlow's SavedModel format
